In [ ]:
!pip install ultralytics

In [1]:
import numpy as np
import pandas as pd
import regex as re
import os
import shutil

from ultralytics import YOLO
from yolo_cam.eigen_cam import EigenCAM
from yolo_cam.utils.image import show_cam_on_image, scale_cam_image

import yaml
from zipfile import ZipFile

In [3]:
# Unzip Dataset
data_folder = "ISVC_Localization"
DATA = "/home/datascience/ISVC_2023_Tutorial_XAI_Few_Shot_on_the_Cloud/SampleDataset(s)/"

In [4]:
if not os.path.exists(os.path.join(DATA, data_folder)):
    with ZipFile(f"{DATA}/{data_folder}.zip", 'r') as zipf:
        zipf.extractall(f"{DATA}")

In [10]:
from sklearn.model_selection import train_test_split
def generate_datasets(root_dir, image_dir, annot_dir,  task="localization"):
    img_dir = os.path.join(root_dir, image_dir)
    annot_dir = os.path.join(root_dir, annot_dir)
    
    # PID regex
    pid_reg = re.compile("9[0-9]{6}")
    
    # Image names + annotation names match
    records = [[pid_reg.findall(img)[-1], img, img if task != "localization" else img.replace(".jpg", ".xml")] 
               for img in os.listdir(img_dir) if ".DS_Store" not in img and ".ipynb_checkpoints" not in img]

    data_records = pd.DataFrame(records, columns=["pid", "images", "masks"])

    train, test = train_test_split(data_records.pid.unique(), test_size=0.5, random_state=42)
    valid, test = train_test_split(test, test_size=0.5, random_state=42)

    train = data_records[data_records.pid.isin(train)].reset_index(drop=True)
    valid = data_records[data_records.pid.isin(valid)].reset_index(drop=True)
    test = data_records[data_records.pid.isin(test)].reset_index(drop=True)

    return train, valid, test

In [11]:
train, valid, test = generate_datasets(f"{DATA}/{data_folder}", "images", "labels")

In [12]:
train

,pid,images,masks
0,9319367,9319367.jpg,9319367.xml
1,9326657,9326657.jpg,9326657.xml
2,9413071,9413071.jpg,9413071.xml
3,9337068,9337068.jpg,9337068.xml
4,9372219,9372219.jpg,9372219.xml
5,9353017,9353017.jpg,9353017.xml
6,9433580,9433580.jpg,9433580.xml
7,9357301,9357301.jpg,9357301.xml
8,9322375,9322375.jpg,9322375.xml
9,9398062,9398062.jpg,9398062.xml


In [ ]:
def get_few_shot_sample(dataset, k=1, random_state=42):
    if k > len(dataset):
        return dataset

    return dataset.sample(k, random_state=random_state).reset_index(drop=True)

In [ ]:
train_few = get_few_shot_sample(train, k=10)
valid_few = get_few_shot_sample(valid, k=10)

In [ ]:
few_shot_dir = "yolov7-main/datasets/localization-10-shot"
if not os.path.exists(few_shot_dir):
    os.makedirs(few_shot_dir)

In [ ]:
os.makedirs(f"{few_shot_dir}/train/images")
os.makedirs(f"{few_shot_dir}/train/labels")

os.makedirs(f"{few_shot_dir}/valid/images")
os.makedirs(f"{few_shot_dir}/valid/labels")

os.makedirs(f"{few_shot_dir}/test/images")
os.makedirs(f"{few_shot_dir}/test/labels")


In [ ]:
for pid in train_few.pid:
    shutil.copy(f"{DATA}/{data_folder}/images/{pid}.jpg", f"{few_shot_dir}/train/images/{pid}.jpg")
    shutil.copy(f"{DATA}/{data_folder}/labels/{pid}.txt", f"{few_shot_dir}/train/labels/{pid}.txt")

In [ ]:
for pid in valid_few.pid:
    shutil.copy(f"{DATA}/{data_folder}/images/{pid}.jpg", f"{few_shot_dir}/valid/images/{pid}.jpg")
    shutil.copy(f"{DATA}/{data_folder}/labels/{pid}.txt", f"{few_shot_dir}/valid/labels/{pid}.txt")

In [ ]:
for pid in test.pid:
    shutil.copy(f"{DATA}/{data_folder}/images/{pid}.jpg", f"{few_shot_dir}/test/images/{pid}.jpg")
    shutil.copy(f"{DATA}/{data_folder}/labels/{pid}.txt", f"{few_shot_dir}/test/labels/{pid}.txt")

In [ ]:
yaml_info = {
    "train": "../train/images",
    "val": "../valid/images",
    "test": "../test/images",
    "nc": 1,
    "names": ['KneeAPView']
}

In [ ]:
with open(f'{few_shot_dir}/data.yaml', 'w') as f:
    yaml.dump(yaml_info, f)

### Few-Shot YOLOv8 Training

In [ ]:
# Import YOLO from Ultralytics library
from ultralytics import YOLO
import os

In [ ]:
model = YOLO("yolov8n.pt")  # load a pretrained model (recommended for training)

In [ ]:
# Train on few-shot dataset
few_shot_root = "/home/datascience/ISVC_2023_Tutorial_XAI_Few_Shot_on_the_Cloud/Code/Localization/yolov7-main/datasets/localization-10-shot/data.yaml"
metrics = model.train(data=few_shot_root, epochs=100)

In [ ]:
metrics = model.val()  # evaluate model performance on the validation set

In [ ]:
# Evaluate on test set
results = model.val(split="test")

In [ ]:
# Single image prediction
result = model.predict("yolov7-main/datasets/localization-10-shot/test/images/9326106.jpg", save=True)

In [ ]:
# Multiple image prediction
test_root = "yolov7-main/datasets/localization-10-shot/test/images/"
test_imgs = [os.path.join(test_root, img) for img in os.listdir(test_root)]
result = model.predict(test_imgs, save=True)

In [ ]:
## Save Model

In [ ]:
!mkdir checkpoints
!mkdir checkpoints/10-shot
!cp runs/detect/train3/weights/best.pt checkpoints/10-shot

In [ ]:
trained_model_pth = "runs/detect/train3/weights/best.pt"
model = YOLO(trained_model_pth)

In [ ]:
target_layers =[model.model.model[-2]]

In [ ]:
cam = EigenCAM(model, target_layers,task='od')

In [ ]:
img = cv2.imread("yolov7-main/datasets/localization-10-shot/test/images/9326106.jpg")

In [ ]:
img.shape

In [ ]:
plt.imshow(img)

In [ ]:
img = cv2.resize(img, (640, 640))
rgb_img = img.copy()
img = np.float32(img) / 255

grayscale_cam = cam(rgb_img)[0, :, :]
cam_image = show_cam_on_image(img, grayscale_cam, use_rgb=True)
plt.imshow(cam_image)
plt.show()

In [ ]:
g_scale = cv2.resize(np.stack([grayscale_cam] * 3, axis=2), (224,224))

plt.imshow(g_scale, cmap='gray')
plt.show()

In [ ]:
# im = cv2.cvtColor(rgb_img, cv2.COLOR_RGB2BGR)
Image.fromarray(np.hstack((rgb_img, cam_image)))